In [1]:
import datashader
import plotly
import bokeh

In [2]:
import sys
sys.path.append('/home/leutloff/Software/generatorpipeline/generatorpipeline/')
import accumulators as acc

In [ ]:
[]

In [3]:
import numpy as np

def generate_images(num_images):
    """
    Erzeugt eine Liste von `num_images` 3D-Bildern mit der Form (16, 512, 128).
    Jedes Bild enthält zufällige Werte zwischen 0 und 255 (als Ganzzahlen).
    
    :param num_images: Die Anzahl der zu erzeugenden Bilder
    :return: Eine Liste von 3D-Bildern mit der Form (16, 512, 128)
    """
    images = []
    for _ in range(num_images):
        # Erzeuge ein Bild mit zufälligen Werten im Bereich [0, 255]
        image = np.random.random(size=(16, 512, 128)).astype(np.float32)
        images.append(image)
    
    return images

In [4]:
def gen(images):
    for image in images:
        yield image

In [5]:
def max_image(generator):
    # Start mit dem ersten Bild im Generator
    max_img_data = next(generator)

    # Gehe durch den Generator und vergleiche jedes Bild
    for image in generator:
        img_data = image
        # Berechne das Maximum der 3D-Daten (alle Dimensionen)
        max_img_data = np.maximum(max_img_data, img_data)
        
    return max_img_data

def max_image2(generator):

    # Start mit dem ersten Bild im Generator
    max_img_data = next(generator)

    # Gehe durch den Generator und vergleiche jedes Bild
    for image in generator:
        img_data = image
        # Berechne die Differenz
        diff = img_data - max_img_data
        # Erstelle eine Maske, wo das Bild größer ist
        mask = diff > 0
        # Aktualisiere das Maximalbild basierend auf der Maske
        max_img_data[mask] = img_data[mask]
        
    return max_img_data

def max_image3(generator):

    # Start mit dem ersten Bild im Generator
    max_img_data = next(generator)

    # Gehe durch den Generator und vergleiche jedes Bild
    for image in generator:
        img_data = image
        # Berechne die Differenz
        diff = img_data - max_img_data
        # Erstelle eine Maske, wo das Bild größer ist
        mask1 = diff > 0
        mask2 = ~mask1
        # Aktualisiere das Maximalbild basierend auf der Maske
        max_img_data = img_data*mask1 + max_img_data*mask2
        
    return max_img_data

def max_image4(generator):

    # Start mit dem ersten Bild im Generator
    max_img_data = acc.Maximum()

    # Gehe durch den Generator und vergleiche jedes Bild
    for image in generator:
        max_img_data.accumulate(image)
        
    return max_img_data.value

def max_10_images(generator):
    max_img_data = acc.CacheMaximum(length=10, key=(lambda x: x[1]))

    for image in generator:
        lit = np.sum(image>0.5)
        max_img_data.accumulate([image, lit])

    return max_img_data.value

In [6]:
images = generate_images(4500)
print(len(images))

4500


In [7]:
image_gen = gen(images)

In [8]:
%%time
max_10_images(image_gen)

CPU times: user 2.66 s, sys: 446 ms, total: 3.1 s
Wall time: 3.13 s


[[array([[[0.93237495, 0.9448161 , 0.9142076 , ..., 0.7784644 ,
           0.73797846, 0.68483853],
          [0.79919493, 0.7059531 , 0.90059334, ..., 0.19538805,
           0.49985495, 0.5691186 ],
          [0.42054388, 0.12431981, 0.5881547 , ..., 0.6321199 ,
           0.36294228, 0.22750257],
          ...,
          [0.51155424, 0.22438203, 0.34636098, ..., 0.72405225,
           0.8168714 , 0.1177829 ],
          [0.37439293, 0.4392759 , 0.39457437, ..., 0.7447469 ,
           0.40645966, 0.17531684],
          [0.5589454 , 0.9555211 , 0.9201623 , ..., 0.6708293 ,
           0.16814303, 0.4388015 ]],
  
         [[0.84477997, 0.84034646, 0.64247286, ..., 0.9874428 ,
           0.6449232 , 0.80479795],
          [0.38750428, 0.8538407 , 0.6878535 , ..., 0.34897402,
           0.9823187 , 0.7382691 ],
          [0.16217753, 0.66451865, 0.3968053 , ..., 0.6222819 ,
           0.5942172 , 0.5475976 ],
          ...,
          [0.44534233, 0.9004139 , 0.3003061 , ..., 0.7883054 ,
  

In [58]:
%%time
max_image4(image_gen)

CPU times: user 2.69 s, sys: 79.7 ms, total: 2.77 s
Wall time: 2.79 s


array([[[0.9994573 , 0.9999619 , 0.99980074, ..., 0.99995965,
         0.99973714, 0.9998859 ],
        [0.99981415, 0.99956656, 0.999741  , ..., 0.9998919 ,
         0.99995226, 0.999981  ],
        [0.9998869 , 0.9999991 , 0.99988693, ..., 0.99995893,
         0.9994444 , 0.99985904],
        ...,
        [0.999865  , 0.99977493, 0.9999391 , ..., 0.99932337,
         0.99988633, 0.9997095 ],
        [0.999974  , 0.9999328 , 0.9999201 , ..., 0.99978524,
         0.99990606, 0.9997642 ],
        [0.99974614, 0.9999556 , 0.9999504 , ..., 0.9997562 ,
         0.99973136, 0.99961114]],

       [[0.9998073 , 0.9994525 , 0.99979717, ..., 0.9998511 ,
         0.99988526, 0.99992234],
        [0.99988955, 0.99965763, 0.9999025 , ..., 0.9997696 ,
         0.99957144, 0.9998792 ],
        [0.99972695, 0.9998775 , 0.9990693 , ..., 0.9990128 ,
         0.9997964 , 0.9998982 ],
        ...,
        [0.99989265, 0.9993602 , 0.9996236 , ..., 0.999682  ,
         0.9998921 , 0.9997008 ],
        [0.9

In [43]:
%%time
max_image(image_gen)

CPU times: user 3.17 s, sys: 8.18 ms, total: 3.18 s
Wall time: 3.48 s


array([[[0.9994573 , 0.9999619 , 0.99980074, ..., 0.99995965,
         0.99973714, 0.9998859 ],
        [0.99981415, 0.99956656, 0.999741  , ..., 0.9998919 ,
         0.99995226, 0.999981  ],
        [0.9998869 , 0.9999991 , 0.99988693, ..., 0.99995893,
         0.9994444 , 0.99985904],
        ...,
        [0.999865  , 0.99977493, 0.9999391 , ..., 0.99932337,
         0.99988633, 0.9997095 ],
        [0.999974  , 0.9999328 , 0.9999201 , ..., 0.99978524,
         0.99990606, 0.9997642 ],
        [0.99974614, 0.9999556 , 0.9999504 , ..., 0.9997562 ,
         0.99973136, 0.99961114]],

       [[0.9998073 , 0.9994525 , 0.99979717, ..., 0.9998511 ,
         0.99988526, 0.99992234],
        [0.99988955, 0.99965763, 0.9999025 , ..., 0.9997696 ,
         0.99957144, 0.9998792 ],
        [0.99972695, 0.9998775 , 0.9990693 , ..., 0.9990128 ,
         0.9997964 , 0.9998982 ],
        ...,
        [0.99989265, 0.9993602 , 0.9996236 , ..., 0.999682  ,
         0.9998921 , 0.9997008 ],
        [0.9

In [46]:
%%time
max_image2(image_gen)

CPU times: user 7.89 s, sys: 17 ms, total: 7.91 s
Wall time: 8.5 s


array([[[0.9994573 , 0.9999619 , 0.99980074, ..., 0.99995965,
         0.99973714, 0.9998859 ],
        [0.99981415, 0.99956656, 0.999741  , ..., 0.9998919 ,
         0.99995226, 0.999981  ],
        [0.9998869 , 0.9999991 , 0.99988693, ..., 0.99995893,
         0.9994444 , 0.99985904],
        ...,
        [0.999865  , 0.99977493, 0.9999391 , ..., 0.99932337,
         0.99988633, 0.9997095 ],
        [0.999974  , 0.9999328 , 0.9999201 , ..., 0.99978524,
         0.99990606, 0.9997642 ],
        [0.99974614, 0.9999556 , 0.9999504 , ..., 0.9997562 ,
         0.99973136, 0.99961114]],

       [[0.9998073 , 0.9994525 , 0.99979717, ..., 0.9998511 ,
         0.99988526, 0.99992234],
        [0.99988955, 0.99965763, 0.9999025 , ..., 0.9997696 ,
         0.99957144, 0.9998792 ],
        [0.99972695, 0.9998775 , 0.9990693 , ..., 0.9990128 ,
         0.9997964 , 0.9998982 ],
        ...,
        [0.99989265, 0.9993602 , 0.9996236 , ..., 0.999682  ,
         0.9998921 , 0.9997008 ],
        [0.9

In [52]:
%%time
max_image3(image_gen)

CPU times: user 18.3 s, sys: 7.26 s, total: 25.5 s
Wall time: 27.4 s


array([[[0.9994573 , 0.9999619 , 0.99980074, ..., 0.99995965,
         0.99973714, 0.9998859 ],
        [0.99981415, 0.99956656, 0.999741  , ..., 0.9998919 ,
         0.99995226, 0.999981  ],
        [0.9998869 , 0.9999991 , 0.99988693, ..., 0.99995893,
         0.9994444 , 0.99985904],
        ...,
        [0.999865  , 0.99977493, 0.9999391 , ..., 0.99932337,
         0.99988633, 0.9997095 ],
        [0.999974  , 0.9999328 , 0.9999201 , ..., 0.99978524,
         0.99990606, 0.9997642 ],
        [0.99974614, 0.9999556 , 0.9999504 , ..., 0.9997562 ,
         0.99973136, 0.99961114]],

       [[0.9998073 , 0.9994525 , 0.99979717, ..., 0.9998511 ,
         0.99988526, 0.99992234],
        [0.99988955, 0.99965763, 0.9999025 , ..., 0.9997696 ,
         0.99957144, 0.9998792 ],
        [0.99972695, 0.9998775 , 0.9990693 , ..., 0.9990128 ,
         0.9997964 , 0.9998982 ],
        ...,
        [0.99989265, 0.9993602 , 0.9996236 , ..., 0.999682  ,
         0.9998921 , 0.9997008 ],
        [0.9